# Model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,precision_score,recall_score
from sklearn import preprocessing
import imblearn
from imblearn.over_sampling import RandomOverSampler

In [2]:
ft=pd.read_csv('Final_Table_mapping.csv')

X=ft.drop(['plan','user_id','age'],axis=1)
y=ft['plan']
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2)

In [3]:
ros=RandomOverSampler()
x_over,y_over=ros.fit_sample(X_train,y_train)
lr=LogisticRegression(solver='liblinear')
scores=cross_val_score(lr,x_over,y_over,cv=10,scoring='accuracy')
lr.fit(x_over,y_over)
y_pred=lr.predict(x_over)
print(classification_report(y_over,y_pred))

              precision    recall  f1-score   support

           0       0.68      0.84      0.75     10929
           1       0.79      0.60      0.68     10929

    accuracy                           0.72     21858
   macro avg       0.73      0.72      0.72     21858
weighted avg       0.73      0.72      0.72     21858



# New sample

In [4]:
t=pd.read_csv('transactions_test.csv')
users=pd.read_csv('users_test.csv')

users_new=users.drop(['country','city','user_settings_crypto_unlocked','attributes_notifications_marketing_push', 'attributes_notifications_marketing_email','num_contacts','num_referrals','num_successful_referrals'], axis=1)

In [5]:
users_new

,user_id,birth_year,created_date
0,user_2,1987,2018-01-18 19:17:31.229096
1,user_8,1981,2018-01-03 07:28:59.063740
2,user_10,1980,2018-01-04 08:05:15.466282
3,user_13,1995,2018-01-28 16:56:16.221271
4,user_17,1996,2018-01-08 04:18:47.595266
...,...,...,...
3881,user_18748,1975,2018-12-05 16:44:40.034848
3882,user_18788,1981,2018-12-13 20:38:52.650550
3883,user_18904,1962,2018-12-17 06:27:44.307084
3884,user_19194,1989,2018-12-16 01:07:20.038358


In [6]:
a=[] 
b=[]
for i in users_new['created_date']:
    a.append(i[0:4])
    b.append(i[5:7])

users_new=users_new.assign(year = a,month = b)

In [7]:
users_n2=users_new.copy()
users_n2=users_n2.drop(['created_date'],axis=1)


In [8]:
c=[] 
for i in users_n2['user_id']:
    c.append(i[5:])

users_n2=users_n2.assign(id1= c)

In [9]:
users_n2=users_n2.drop(['user_id'],axis=1)

In [10]:
users_n2 = users_n2.rename({'id1': 'user_id'}, axis=1)

In [11]:
d=[] 
for i in users_n2['birth_year']:
    d.append(2020-int(i))
    
users_n3=users_n2.copy()
users_n3=users_n3.assign(age = d)
users_n4=users_n3.drop(['birth_year'],axis=1)

In [12]:
f=[] 
for i in t['user_id']:
    f.append(i[5:])

t=t.assign(id1= f)

In [13]:
t=t.drop(['user_id'],axis=1)
t = t.rename({'id1': 'user_id'}, axis=1)

In [14]:
t=t[t['transactions_state']=='COMPLETED']

In [15]:
u=[]
x=0
for i in t['user_id']:
    if x==0:
        u.append(i)
        x=x+1
    elif i not in u:
        u.append(i)
    else:continue

In [16]:
u1=[]
for i in u:
    i1=int(i)
    u1.append(i1)
u1=sorted(u1)

In [17]:
users_n4['user_id']=users_n4['user_id'].astype(int)
h=users_n4['user_id']

In [18]:
u_n2=[]
for i in h:
    if i not in u1:
        u_n2.append(i)
for i in u_n2:
    i=int(i)



In [19]:
users_n4=users_n4.set_index('user_id')

In [20]:
for i in u_n2:
    users_n4=users_n4.drop([i])

In [21]:
users_n5=users_n4.reset_index()


In [22]:
k=[] 
l=[]
for i in t['created_date']:
    k.append(i[0:4])
    l.append(i[5:7])

t=t.assign(year = k,month = l)
t=t.drop(['created_date'],axis=1)

In [23]:
t['user_id']=t['user_id'].astype(int)
pr=[]
for i in u1:
    df=t[t['user_id']==i]
    x=df['amount_usd'].sum()
    pr.append(x)

In [24]:
users_n5=users_n5.assign(monetary_value = pr)

In [25]:
t['user_id']=t['user_id'].astype(int)
fr=[]
x=0
for i in u1:
    df=t[t['user_id']==i]
    x=len(df['amount_usd'])
    fr.append(x)

In [26]:
users_n6=users_n5.assign(frequency_value = fr)

In [27]:
age_dict={'2018':0,'2019':12}
users_n6['year']=users_n6['year'].map(age_dict)

In [28]:
users_n6['year']=users_n6['year'].astype(int)
users_n6['month']=users_n6['month'].astype(int)
        

In [29]:
users_n7=users_n6.copy()
users_n7['active']=17-(users_n7['year']+users_n7['month'])

In [30]:
users_n7['monetary_value']=users_n7['monetary_value']/users_n7['active']
users_n7['frequency_value']=users_n7['frequency_value']/users_n7['active']

In [31]:
age_dict={'2018':0,'2019':12}
t['year']=t['year'].map(age_dict)
t['month']=t['month'].astype(int)
tt=t.copy()
tt['touches']=tt['year']+tt['month']

In [32]:
tt1=tt.sort_values(['user_id', 'year','month'])

In [33]:
x=users_n7['user_id']
x1=[]
for i in x:
    i1=int(i)
    x1.append(i1)
    
re=[]
for j in x1:
    df=tt1[tt1['user_id']==j]
    y=df.iloc[-1,-1]
    re.append(y)

In [34]:
users_n7=users_n7.assign(touches = re)

In [35]:
users_n7['creation']=17
users_n7['recency_value']=users_n7['creation']-users_n7['touches']


In [36]:
users_n7=users_n7.drop(['year','month','creation','touches'],axis=1)

In [37]:
users_n8=users_n7.drop(['active'],axis=1)

In [38]:
m={16:1,15:1,14:1,13:1,12:1,11:1,10:1,9:1,8:1,7:2,6:2,5:2,4:2,3:2,2:3,1:3,0:3}
users_n8["recency_value"]=users_n8["recency_value"].map(m)

In [39]:
users_n8

,user_id,age,monetary_value,frequency_value,recency_value
0,2,33,64.783750,5.0625,3
1,8,39,21.130625,0.9375,1
2,10,40,1530.816875,52.6875,3
3,13,25,78.676875,4.1250,3
4,17,24,1236.003750,2.0000,2
...,...,...,...,...,...
3696,18748,45,14.396000,1.2000,3
3697,18788,39,767.566000,38.2000,3
3698,18904,58,638.418000,41.4000,3
3699,19194,31,3.094000,1.0000,3


## prediction

In [40]:
s=users_n8.drop(['user_id','age'],axis=1)

In [41]:
lr.predict(s)

array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [42]:
len(u_n2)

185

In [43]:
final1=pd.DataFrame()

In [44]:
final1['user_id']=users_n8['user_id']
final1['plan_prediction']=lr.predict(s)


In [45]:
final2=pd.DataFrame()

In [46]:
final2['user_id']=u_n2
final2['plan_prediction']=0

In [47]:
final = pd.concat([final1, final2])
final

,user_id,plan_prediction
0,2,0
1,8,0
2,10,1
3,13,0
4,17,0
...,...,...
180,19267,0
181,19273,0
182,19276,0
183,19363,0


In [48]:
final=final.sort_values(by='user_id')
final


,user_id,plan_prediction
0,2,0
1,8,0
2,10,1
3,13,0
4,17,0
...,...,...
3700,19394,0
3409,19402,1
3410,19407,1
3411,19416,1


In [49]:
final['plan_prediction'].value_counts()

0    3069
1     817
Name: plan_prediction, dtype: int64

In [50]:
p=[]
for i in final['user_id']:
    i1=str(i)
    p.append(i1)

In [51]:
w=[]
for i in p:
    z='user_'+i
    w.append(z)

In [52]:
final['user_id']=w

In [53]:
final

,user_id,plan_prediction
0,user_2,0
1,user_8,0
2,user_10,1
3,user_13,0
4,user_17,0
...,...,...
3700,user_19394,0
3409,user_19402,1
3410,user_19407,1
3411,user_19416,1


In [54]:
final.to_csv('test_preds.csv', index=False)